# Naive Bayes Classifier - Automobile Accidents

This program is a solution to the problem 8.2 of chapter 8 of following book. 

Data Mining for Business Analytics: Concepts, Techniques, and Applications in Python, First Edition.

Galit Shmueli, Peter C. Bruce, Peter Gedeck, and Nitin R. Patel

© 2020 John Wiley & Sons, Inc. Published 2020 by John Wiley & Sons, Inc.

##  Chapter 8, Problem 8.2

8.2 Automobile Accidents. The file accidentsFull.csv contains information on 42,183 actual automobile accidents in 2001 in the United States that involved one of three levels of injury: NO INJURY, INJURY, or FATALITY. For each accident, additional information is recorded, such as day of week, weather conditions, and road type. A firm might be interested in developing a system for quickly classifying the severity of an accident based on initial reports and associated data in the system (some of which rely on GPS-assisted reporting).

Our goal here is to predict whether an accident just reported will involve an injury (MAX_SEV_IR = 1 or 2) or will not (MAX_SEV_IR = 0). For this purpose, create a dummy variable called INJURY that takes the value “yes” if MAX_SEV_IR = 1 or 2, and otherwise “no.” 

a. Using the information in this dataset, if an accident has just been reported and no further information is available, what should the prediction be? (INJURY = Yes or No?) Why?

b. Select the first 12 records in the dataset and look only at the response (INJURY) and the two predictors WEATHER_R and TRAF_CON_R.

    i. Create a pivot table that examines INJURY as a function of the two predictors for these 12 records. Use all three variables in the pivot table as rows/columns.
    ii. Compute the exact Bayes conditional probabilities of an injury (INJURY = Yes) given the six possible combinations of the predictors.
    iii. Classify the 12 accidents using these probabilities and a cutoff of 0.5.
    iv. Compute manually the naive Bayes conditional probability of an injury given WEATHER_R = 1 and TRAF_CON_R = 1.
    v. Run a naive Bayes classifier on the 12 records and 2 predictors using scikitlearn. Check the model output to obtain probabilities and classifications for all 12 records. Compare this to the exact Bayes classification. Are the resulting classifications equivalent? Is the ranking (=ordering) of observations equivalent?
    
c. Let us now return to the entire dataset. Partition the data into training (60%) and validation (40%).

    i. Assuming that no information or initial reports about the accident itself are available at the time of prediction (only location characteristics, weather conditions, etc.), which predictors can we include in the analysis? (Use the data descriptions page from www.dataminingbook.com.)
    ii. Run a naive Bayes classifier on the complete training set with the relevant predictors (and INJURY as the response). Note that all predictors are categorical. Show the confusion matrix.
    iii. What is the overall error for the validation set?
    iv. What is the percent improvement relative to the naive rule (using the validation set)?
    v. Examine the conditional probabilities in the pivot tables. Why do we get a probability of zero for P(INJURY = No | SPD_LIM = 5)?

## Importing Libraries

In [238]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from dmba import classificationSummary

Printing versions of libraries

In [239]:
print('pandas version: {}'.format(pd.__version__))
print('numpy version: {}'.format(np.__version__))
print('sklearn version: {}'.format(sklearn.__version__))

pandas version: 1.5.3
numpy version: 1.23.5
sklearn version: 1.2.1


## Loading Dataset

In [240]:
df = pd.read_csv('AccidentsFull.csv')
print(df.shape)
display(df.head())

(42183, 24)


,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
0,0,2,2,1,0,1,0,3,0,0,...,4,0,3,1,1,1,1,0,0,1
1,1,2,1,0,0,1,1,3,2,0,...,4,0,3,2,2,0,0,1,0,0
2,1,2,1,0,0,1,0,3,2,0,...,4,1,2,2,2,0,0,1,0,0
3,1,2,1,1,0,0,0,3,2,0,...,4,1,2,2,1,0,0,1,0,0
4,1,1,1,0,0,1,0,3,2,0,...,4,0,2,3,1,0,0,1,0,0


##### Creating a dummy variable called INJURY that takes the value “yes” if MAX_SEV_IR = 1 or 2, and otherwise “no.”

In [241]:
df['INJURY'] = np.where(df['MAX_SEV_IR'].isin([1, 2]), 'yes', 'no')
df.head()

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR,INJURY
0,0,2,2,1,0,1,0,3,0,0,...,0,3,1,1,1,1,0,0,1,yes
1,1,2,1,0,0,1,1,3,2,0,...,0,3,2,2,0,0,1,0,0,no
2,1,2,1,0,0,1,0,3,2,0,...,1,2,2,2,0,0,1,0,0,no
3,1,2,1,1,0,0,0,3,2,0,...,1,2,2,1,0,0,1,0,0,no
4,1,1,1,0,0,1,0,3,2,0,...,0,2,3,1,0,0,1,0,0,no


##### a. Using the information in this dataset, if an accident has just been reported and no further information is available, what should the prediction be? (INJURY = Yes or No?) Why?

In [242]:
count_total_accidents = len(df.index)
count_injury_yes = len(df[df['INJURY'] ==  'yes'].index)
count_injury_no = len(df[df['INJURY'] == 'no'].index)

print('Total Accidents: ', count_total_accidents)
print("Count of INJURY(INJURY = 'yes')", count_injury_yes)
print("Count of INJURY(INJURY = 'no')", count_injury_no)

Total Accidents:  42183
Count of INJURY(INJURY = 'yes') 21462
Count of INJURY(INJURY = 'no') 20721


In [243]:
prob_injury_yes = count_injury_yes/count_total_accidents
prob_injury_no = count_injury_no/count_total_accidents

print('P(INJURY = Yes):', prob_injury_yes)
print('P(INJURY = No):', prob_injury_no)

P(INJURY = Yes): 0.5087831590925255
P(INJURY = No): 0.4912168409074746


###### Result

If an accident has just been reported and no further information is available, we will use the rule "assign to the most probable class". In this case since P(INJURY = Yes) is greater than P(INJURY = No), the most probable class is 'INJURY = Yes'. Therefore, the prediction will be INJURY = Yes. 

##### b. Select the first 12 records in the dataset and look only at the response (INJURY) and the two predictors WEATHER_R and TRAF_CON_R.

###### i. Create a pivot table that examines INJURY as a function of the two predictors for these 12 records. Use all three variables in the pivot table as rows/columns.

In [244]:
df_12_rec = df[['TRAF_CON_R', 'WEATHER_R', 'INJURY']].iloc[:12,]
display(df_12_rec)
pivot_table = df_12_rec.pivot_table(index='INJURY', columns=['TRAF_CON_R', 'WEATHER_R'], aggfunc='size', fill_value=0)
display(pivot_table)

,TRAF_CON_R,WEATHER_R,INJURY
0,0,1,yes
1,0,2,no
2,1,2,no
3,1,1,no
4,0,1,no
5,0,2,yes
6,0,2,no
7,0,1,yes
8,0,2,no
9,0,2,no


TRAF_CON_R  0     1     2
WEATHER_R   1  2  1  2  1
INJURY                   
no          1  5  1  1  1
yes         2  1  0  0  0

###### ii. Compute the exact Bayes conditional probabilities of an injury (INJURY = Yes) given the six possible combinations of the predictors.

In [245]:
injury_yes_exact_bayes_df = pd.DataFrame({'TRAF_CON_R': [],
                               'WEATHER_R': [],
                               'PROB_INJURY_YES': []})
prob_yes_0_1 = 2/3
print('P(INJURY = Yes | TRAF_CON_R = 0, WEATHER_R = 1) = ', prob_yes_0_1)
injury_yes_exact_bayes_df.loc[len(injury_yes_exact_bayes_df)] = [0, 1, prob_yes_0_1]

prob_yes_0_2 = 1/6
print('P(INJURY = Yes | TRAF_CON_R = 0, WEATHER_R = 2) = ', prob_yes_0_2)
injury_yes_exact_bayes_df.loc[len(injury_yes_exact_bayes_df)] = [0, 2, prob_yes_0_2]

prob_yes_1_1 = 0/1
print('P(INJURY = Yes | TRAF_CON_R = 1, WEATHER_R = 1) = ', prob_yes_1_1)
injury_yes_exact_bayes_df.loc[len(injury_yes_exact_bayes_df)] = [1, 1, prob_yes_1_1]

prob_yes_1_2 = 0/1
print('P(INJURY = Yes | TRAF_CON_R = 1, WEATHER_R = 2) = ', prob_yes_1_2)
injury_yes_exact_bayes_df.loc[len(injury_yes_exact_bayes_df)] = [1, 2, prob_yes_1_2]

prob_yes_2_1 = 0/1
print('P(INJURY = Yes | TRAF_CON_R = 2, WEATHER_R = 1) = ', prob_yes_2_1)
injury_yes_exact_bayes_df.loc[len(injury_yes_exact_bayes_df)] = [2, 1, prob_yes_2_1]

prob_yes_2_2 = 0 # 0/0
print('P(INJURY = Yes | TRAF_CON_R = 2, WEATHER_R = 2) = ', prob_yes_2_2)
injury_yes_exact_bayes_df.loc[len(injury_yes_exact_bayes_df)] = [2, 2, prob_yes_2_2]

injury_yes_exact_bayes_df['TRAF_CON_R'] = injury_yes_exact_bayes_df['TRAF_CON_R'].astype(int)
injury_yes_exact_bayes_df['WEATHER_R'] = injury_yes_exact_bayes_df['WEATHER_R'].astype(int)
display(injury_yes_exact_bayes_df)

P(INJURY = Yes | TRAF_CON_R = 0, WEATHER_R = 1) =  0.6666666666666666
P(INJURY = Yes | TRAF_CON_R = 0, WEATHER_R = 2) =  0.16666666666666666
P(INJURY = Yes | TRAF_CON_R = 1, WEATHER_R = 1) =  0.0
P(INJURY = Yes | TRAF_CON_R = 1, WEATHER_R = 2) =  0.0
P(INJURY = Yes | TRAF_CON_R = 2, WEATHER_R = 1) =  0.0
P(INJURY = Yes | TRAF_CON_R = 2, WEATHER_R = 2) =  0


,TRAF_CON_R,WEATHER_R,PROB_INJURY_YES
0,0,1,0.666667
1,0,2,0.166667
2,1,1,0.000000
3,1,2,0.000000
4,2,1,0.000000
5,2,2,0.000000


###### iii. Classify the 12 accidents using these probabilities and a cutoff of 0.5.

In [246]:
df_merged = df_12_rec.merge(injury_yes_exact_bayes_df, on=["TRAF_CON_R", "WEATHER_R"])
df_merged['CLASSIFICATION'] = 'no'
df_merged.loc[df_merged["PROB_INJURY_YES"] > 0.5, "CLASSIFICATION"] = 'yes'
display(df_merged)

,TRAF_CON_R,WEATHER_R,INJURY,PROB_INJURY_YES,CLASSIFICATION
0,0,1,yes,0.666667,yes
1,0,1,no,0.666667,yes
2,0,1,yes,0.666667,yes
3,0,2,no,0.166667,no
4,0,2,yes,0.166667,no
5,0,2,no,0.166667,no
6,0,2,no,0.166667,no
7,0,2,no,0.166667,no
8,0,2,no,0.166667,no
9,1,2,no,0.000000,no


###### iv. Compute manually the naive Bayes conditional probability of an injury given WEATHER_R = 1 and TRAF_CON_R = 1.

In [247]:
print('P(INJURY = yes | WEATHER_R = 1, TRAF_CON_R = 1) =\n')

print('                      P(WEATHER_R = 1 | INJURY = yes) P(TRAF_CON_R = 1 | INJURY = yes) P(INJURY = yes)')
print('= -----------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('   P(WEATHER_R = 1 | INJURY = yes) P(TRAF_CON_R = 1 | INJURY = yes) P(INJURY = yes) + P(WEATHER_R = 1 | INJURY = no) P(TRAF_CON_R = 1 | INJURY = no) P(INJURY = no)')

P(INJURY = yes | WEATHER_R = 1, TRAF_CON_R = 1) =

                      P(WEATHER_R = 1 | INJURY = yes) P(TRAF_CON_R = 1 | INJURY = yes) P(INJURY = yes)
= -----------------------------------------------------------------------------------------------------------------------------------------------------------------
   P(WEATHER_R = 1 | INJURY = yes) P(TRAF_CON_R = 1 | INJURY = yes) P(INJURY = yes) + P(WEATHER_R = 1 | INJURY = no) P(TRAF_CON_R = 1 | INJURY = no) P(INJURY = no)


In [248]:
# Obtaining following values from above pivot table
p_W_1_I_yes = 2/3
p_T_1_I_yes = 0/3
p_I_yes = 3/12
p_W_1_I_no = 3/9
p_T_1_I_no = 2/9
p_I_no = 9/12

try:
    prob = (p_W_1_I_yes * p_T_1_I_yes * p_I_yes)/((p_W_1_I_yes * p_T_1_I_yes * p_I_yes) + (p_W_1_I_no * p_T_1_I_no * p_I_no))
except ZeroDivisionError:
    prob = 0

print('P(INJURY = yes | WEATHER_R = 1, TRAF_CON_R = 1) = ', prob)

P(INJURY = yes | WEATHER_R = 1, TRAF_CON_R = 1) =  0.0


###### v. Run a naive Bayes classifier on the 12 records and 2 predictors using scikitlearn. Check the model output to obtain probabilities and classifications for all 12 records. Compare this to the exact Bayes classification. Are the resulting classifications equivalent? Is the ranking (=ordering) of observations equivalent?

Converting variables to categorical.

In [249]:
df_12_rec.TRAF_CON_R = df_12_rec.TRAF_CON_R.astype('category')
df_12_rec.WEATHER_R = df_12_rec.WEATHER_R.astype('category')
df_12_rec.INJURY = df_12_rec.INJURY.astype('category')

In [250]:
X = pd.get_dummies(df_12_rec[['TRAF_CON_R', 'WEATHER_R']])
y = df_12_rec['INJURY']
classes = list(y.cat.categories)

Running naive Bayes 

In [251]:
nb = MultinomialNB(alpha=0.01)
nb.fit(X, y)

MultinomialNB(alpha=0.01)

Predicting probabilities 

In [252]:
predProb = nb.predict_proba(X)

Predicting class membership

In [253]:
y_pred = nb.predict(X)

In [254]:
temp_df = pd.concat([df_12_rec,
                     pd.DataFrame({'actual': y, 'predicted': y_pred}),
                     pd.DataFrame(predProb, index=y.index)], axis=1)
 
display(temp_df)

,TRAF_CON_R,WEATHER_R,INJURY,actual,predicted,0,1
0,0,1,yes,yes,no,0.501932,0.498068
1,0,2,no,no,no,0.800176,0.199824
2,1,2,no,no,no,0.997525,0.002475
3,1,1,no,no,no,0.990239,0.009761
4,0,1,no,no,no,0.501932,0.498068
5,0,2,yes,yes,no,0.800176,0.199824
6,0,2,no,no,no,0.800176,0.199824
7,0,1,yes,yes,no,0.501932,0.498068
8,0,2,no,no,no,0.800176,0.199824
9,0,2,no,no,no,0.800176,0.199824


Result 

Resulting classifications are more or less equivalent.

##### c. Let us now return to the entire dataset. Partition the data into training (60%) and validation (40%).

###### i. Assuming that no information or initial reports about the accident itself are available at the time of prediction (only location characteristics, weather conditions, etc.), which predictors can we include in the analysis? (Use the data descriptions page from www.dataminingbook.com.)

Considering no information or initial reports about the accident itself are available at the time of prediction (only location characteristics, weather conditions, etc), I think, we can include following predictors in the analysis. 

    1. HOUR_I_R
    2. ALIGN_I
    3. WRK_ZONE
    4. WKDY_I_R
    5. INT_HWY
    6. LGTCON_I_R
    7. RELJCT_I_R
    8. REL_RWY_R
    9. SPD_LIM
    10. SUR_COND
    11. WEATHER_R

In [255]:
predictors = ['HOUR_I_R', 'ALIGN_I', 'WRK_ZONE', 'WKDY_I_R', 'INT_HWY', 'LGTCON_I_R', 'RELJCT_I_R', 'REL_RWY_R', 'SPD_LIM', 'SUR_COND', 'WEATHER_R']

In [256]:
X = df[predictors]
X = X.astype('category')
display(X.head())
X = pd.get_dummies(X)
y = df['INJURY'].astype('category')
classes = list(y.cat.categories)

,HOUR_I_R,ALIGN_I,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,RELJCT_I_R,REL_RWY_R,SPD_LIM,SUR_COND,WEATHER_R
0,0,2,0,1,0,3,1,0,40,4,1
1,1,1,0,1,1,3,1,1,70,4,2
2,1,1,0,1,0,3,1,1,35,4,2
3,1,1,0,0,0,3,1,1,35,4,1
4,1,1,0,1,0,3,0,1,25,4,1


Partitioning the data into training (60%) and validation (40%)

In [257]:
# split into training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

###### ii. Run a naive Bayes classifier on the complete training set with the relevant predictors (and INJURY as the response). Note that all predictors are categorical. Show the confusion matrix.

In [258]:
# run naive Bayes 
delays_nb = MultinomialNB(alpha=0.01)
delays_nb.fit(X_train, y_train)

MultinomialNB(alpha=0.01)

In [259]:
#predict class membership
y_train_pred = delays_nb.predict(X_train)
y_valid_pred = delays_nb.predict(X_valid)

In [260]:
# training 
classificationSummary(y_train, y_train_pred, class_names=classes)
print('\n')

# validation
classificationSummary(y_valid, y_valid_pred, class_names=classes)

Confusion Matrix (Accuracy 0.5370)

       Prediction
Actual   no  yes
    no 4754 7638
   yes 4079 8838


Confusion Matrix (Accuracy 0.5321)

       Prediction
Actual   no  yes
    no 3130 5199
   yes 2696 5849


###### iii. What is the overall error for the validation set?

In [261]:
#overall error = (false positive + false negative) / (true positive + false positive + true negative + false negative)
# or
#overall error = 1 - Accuracy

overall_error = (5199 + 2696)/(3130 + 2696 + 5849 + 5199)
print(round(overall_error, 4))

0.4679


###### iv. What is the percent improvement relative to the naive rule (using the validation set)?

Calculating P(INJURY = Yes) and P(INJURY = No) using naive rule on validation set

In [262]:
count_total_accidents = len(y_valid)
count_injury_yes = y_valid.value_counts()['yes']
count_injury_no = y_valid.value_counts()['no'] 

print('Total Accidents: ', count_total_accidents)
print("Count of INJURY(INJURY = 'yes')", count_injury_yes)
print("Count of INJURY(INJURY = 'no')", count_injury_no)

Total Accidents:  16874
Count of INJURY(INJURY = 'yes') 8545
Count of INJURY(INJURY = 'no') 8329


In [263]:
prob_injury_yes = count_injury_yes/count_total_accidents
prob_injury_no = count_injury_no/count_total_accidents

print('P(INJURY = yes):', prob_injury_yes)
print('P(INJURY = no):', prob_injury_no)

P(INJURY = yes): 0.5064003792817352
P(INJURY = no): 0.4935996207182648


We know, the naive rule is a simple classification strategy that assigns all observations to the majority class, regardless of the values of the predictors. From above calculation we can see P(INJURY = yes) is higher than P(INJURY = no). Since P(INJURY = yes) is a majority class here, all the observations will be predicted as INJURY = Yes. Let's calculate the classification error from naive rule. 

In [264]:
value = 'yes'
y_valid_pred_naive = pd.Series(value, index=range(len(y_valid)))

In [265]:
# classification summary on classification using naive rule
classificationSummary(y_valid, y_valid_pred_naive, class_names=classes)

Confusion Matrix (Accuracy 0.5064)

       Prediction
Actual   no  yes
    no    0 8329
   yes    0 8545


In [266]:
#overall error = 1 - Accuracy
error_naive = 1 - 0.5064
print(round(error_naive, 4))

0.4936


In [267]:
#percent improvement = (error_naive - error_classifier) / error_naive * 100
percent_improvement = (error_naive - overall_error) / (error_naive * 100)
print('Percent improvement relative to the naive rule (using the validation set): ', round(percent_improvement, 4))

Percent improvement relative to the naive rule (using the validation set):  0.0005


###### v. Examine the conditional probabilities in the pivot tables. Why do we get a probability of zero for P(INJURY = No | SPD_LIM = 5)?

In [268]:
temp_df = df[['SPD_LIM', 'INJURY']]
pivot_table = temp_df.pivot_table(index='SPD_LIM', columns=['INJURY'], aggfunc='size')
display(pivot_table)

INJURY,no,yes
SPD_LIM,,
5,2,4
10,11,11
15,93,90
20,159,92
25,2245,1960
30,1807,1908
35,3994,4547
40,1978,2326
45,3240,3347


In [269]:
print('P(INJURY = No | SPD_LIM = 5) = ', round(2/6,4))

P(INJURY = No | SPD_LIM = 5) =  0.3333


We will get a probability of zero for P(INJURY = No | SPD_LIM = 5) if there were no cases in the data where the speed limit is 5 and the injury is no. However, in our case, with data in hand, we do have cases with injury = no where speed limit is 5, therefore, our P(INJURY = No | SPD_LIM = 5) is not zero.